In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
file_path="C:/Users/rafae/OneDrive/Desktop/C19/charity_data.csv"
application_df = pd.read_csv(file_path)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"],1)
application_df.head()

C:\Users\rafae\AppData\Local\Temp/ipykernel_144776/1309730736.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(["EIN", "NAME"],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
app_type_count = application_df.APPLICATION_TYPE.value_counts()
app_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

Change bin size for APPLICATION_TYPE

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_count[app_type_count < 1000].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_count = application_df.CLASSIFICATION.value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

Change bin size for CLASSIFICATION

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_count[class_count < 4000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          6
AFFILIATION               6
CLASSIFICATION            4
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\rafae\AppData\Local\Temp/ipykernel_144776/2023052568.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(application_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Deliverable 3: Optimize the Model
### Attempt # 1-Accuracy to beat: 0.6554

Changed bin sizes for APPLICATION_TYPE and CLASSIFICATION

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 312       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 363
Trainable params: 363
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 28585.3965 - accuracy: 0.4676
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 519.2679 - accuracy: 0.4818
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.7914 - accuracy: 0.5324
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.7244 - accuracy: 0.5324
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6980 - accuracy: 0.5324
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6919 - accuracy: 0.5324
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - 

804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 85/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 

Attempt #1 - Accuracy: 0.5166

# Attempt #2

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 3

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 312       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 3)                 18        
                                                                 
 dense_6 (Dense)             (None, 1)                 4         
                                                                 
Total params: 379
Trainable params: 379
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn_2.fit(X_train,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 6s 6ms/step - loss: 0.6914 - accuracy: 0.5311
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 82/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 88/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 

### Attempt #2 - Accuracy:0.5324

## Attempt # 3

#### Change activation function for hidden layers

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 3

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="linear")
)
# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 8)                 312       
                                                                 
 dense_8 (Dense)             (None, 5)                 45        
                                                                 
 dense_9 (Dense)             (None, 3)                 18        
                                                                 
 dense_10 (Dense)            (None, 1)                 4         
                                                                 
Total params: 379
Trainable params: 379
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn_3.fit(X_train,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.7165 - accuracy: 0.4977
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6910 - accuracy: 0.5324
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6911 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 86/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6911 - accuracy: 0.5324
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 

### Attempt #3 - Accuracy:  0.5476

## Attempt #4

### Change number of neurons and activation functions from original model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 8

nn_4 = tf.keras.models.Sequential()

# First hidden layer
nn_4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 10)                390       
                                                                 
 dense_12 (Dense)            (None, 8)                 88        
                                                                 
 dense_13 (Dense)            (None, 1)                 9         
                                                                 
Total params: 487
Trainable params: 487
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoint/",exist_ok=True)
checkpoint_path = "checkpoint/weights.{epoch:02d}.hdf5"

In [24]:
# Compile the model
nn_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4020)

In [25]:
# Train the model
fit_model = nn_4.fit(X_train,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.6915 - accuracy: 0.5323
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6914 - accuracy: 0.5324
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 5/100
797/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5326
Epoch 5: saving model to checkpoint\weights.05.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 7/100
804/804 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 9/100
804/804 [==============================] - 

795/804 [============================>.] - ETA: 0s - loss: 0.6912 - accuracy: 0.5331
Epoch 65: saving model to checkpoint\weights.65.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 66/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 67/100
804/804 [==============================] - 6s 7ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 68/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 69/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6912 - accuracy: 0.5324
Epoch 70/100
797/804 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5326
Epoch 70: saving model to checkpoint\weights.70.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 71/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6913 - accuracy: 0.5324
Epoch 72/100
804/804 [=

#### Attempt #4 - Accuracy: 0.4484

In [26]:
# Export our model to HDF5 file
nn_4.save("AlphabetSoupCharity_Optimization.h5")